

```
 1. Install required packages
```



In [ ]:
!pip install pathway bokeh panel geopy --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Upload and load your dataset

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import panel as pn
import bokeh.plotting
from google.colab import files
from geopy.distance import geodesic

uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])


Saving modified_dataset.csv to modified_dataset.csv


 3. Preprocess for demand & pricing

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
traffic_weights = {'low': 0.2, 'average': 0.5, 'high': 1.0}

df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weights).fillna(1.0)
df['TrafficScore'] = df['TrafficConditionNearby'].map(traffic_weights).fillna(0.5)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
df['Demand'] = 0.4 * df['OccupancyRate'] + 0.2 * df['QueueNorm'] + 0.2 * df['IsSpecialDay'] + \
               0.1 * df['VehicleTypeWeight'] - 0.1 * df['TrafficScore']
df['NormalizedDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())


4. Simulate demand-based pricing

In [ ]:
df['BasePrice'] = 10
df['DemandPrice'] = df['BasePrice'] * (1 + 1.2 * df['NormalizedDemand'])
df['DemandPrice'] = df['DemandPrice'].clip(5, 20)


5. Add competition logic (proximity & final price)

In [ ]:
# Apply competitive adjustment on all timestamps
def adjust_price_with_neighbors(row, full_df):
    current_loc = (row['Latitude'], row['Longitude'])

    # Get records from other lots at same time
    same_time_df = full_df[full_df['Timestamp'] == row['Timestamp']]
    nearby_prices = []

    for _, other in same_time_df.iterrows():
        if row['SystemCodeNumber'] == other['SystemCodeNumber']:
            continue
        distance = geodesic(current_loc, (other['Latitude'], other['Longitude'])).meters
        if distance < 500:
            nearby_prices.append(other['DemandPrice'])

    if nearby_prices:
        nearby_avg = np.mean(nearby_prices)
        if row['DemandPrice'] > nearby_avg:
            return max(nearby_avg - 1, 5)
        else:
            return min(row['DemandPrice'] + 0.5, 20)
    else:
        return row['DemandPrice']

# Apply to full dataframe
df['FinalPrice'] = df.apply(lambda row: adjust_price_with_neighbors(row, df), axis=1)

# Save full time-series for streaming
df[['SystemCodeNumber', 'Timestamp', 'FinalPrice']].to_csv('competitive_price.csv', index=False)


6. Pathway streaming setup

In [ ]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    FinalPrice: float

data = pw.demo.replay_csv("competitive_price.csv", schema=ParkingSchema, input_rate=100)

fmt = "%Y-%m-%d %H:%M:%S"
stream = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

windowed = stream.windowby(
    pw.this.t,
    instance=pw.this.SystemCodeNumber + '_' + pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t=pw.this._pw_window_end,
    sum_price=pw.reducers.sum(pw.this.FinalPrice),
    count_price=pw.reducers.count()
).with_columns(
    avg_price = pw.this.sum_price / pw.this.count_price
)


7. Real-time visualization

In [ ]:
pn.extension()

def plot_price(source):
    fig = bokeh.plotting.figure(height=400, width=800, x_axis_type='datetime', title='Model 3: Competitive Price')
    fig.line('t', 'avg_price', source=source, line_width=2, color='green')
    fig.circle('t', 'avg_price', source=source, size=6, color='black')
    return fig

viz = windowed.plot(plot_price, sorting_col='t')
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

8. Run the pipeline

In [ ]:
%%capture --no-display
pw.run()


Output()